# Reverse Engineering The Performance of the Beta-1 Turbopump Unit

> Author: Elias Aoubala

> Date: 11/05/2025

In [2]:
import handcalcs.render
import numpy as np
from CoolProp.CoolProp import PropsSI

## 1 - Background

This document contains the authors attempts to reverse engineer the performance of the Beta-1 Turbopump unit based on [this publication](https://apps.dtic.mil/sti/tr/pdf/ADA473854.pdf).

The main aim here is to figure out what the expected efficiency of the turbopump unit was and against our current estimates for the overall system efficiencies.

## 2 - Key Inputs

This section contains all the key inputs that relevant to the derivation of the turbopump units performance.

In [4]:
LB_TO_KG = 0.453592
PSI_TO_PA = 6894.76

### 2.1 - Gas Generator and Turbine

In [7]:
%%render PARAM
m_dot_gg = 0.63 * LB_TO_KG # kg/s
P_c = 290 * PSI_TO_PA # Pa
T_o = 470 + 273.15 # K
C_p = PropsSI("C", "P", P_c, "T", T_o, "Oxygen")
gamma = PropsSI("C", "P", P_c, "T", T_o, "Oxygen")/PropsSI("O", "P", P_c, "T", T_o, "Oxygen")


<IPython.core.display.Latex object>

### 2.2 - Fuel Pump

In [17]:
%%render param

rho_f = 792 # kg/m3
m_dot_f = 2.52 *LB_TO_KG # kg/s
p_in_f = 17 * PSI_TO_PA # Pa
p_out_f = 390 * PSI_TO_PA # Pa

<IPython.core.display.Latex object>

### 2.3 - Oxidiser Pump

In [18]:
%%render param

rho_o = 1100 # kg/m3
m_dot_o = 7.63 *LB_TO_KG # kg/s
p_in_o = 17 * PSI_TO_PA # Pa
p_out_o = 420 * PSI_TO_PA # Pa

<IPython.core.display.Latex object>

## 3 - Performance estimation

In [10]:
%%render

p_fuel = ((p_out_f - p_in_f) * (m_dot_f/rho_f))/1e3

p_oxidiser = ((p_out_o - p_in_o) * (m_dot_o/rho_o))/1e3

p_gg = (m_dot_gg * C_p  * T_o * (1 - ((1e5)/P_c)**((gamma -1 )/gamma)))/1e3

eta_o = (p_fuel + p_oxidiser)/p_gg

<IPython.core.display.Latex object>

We then assume the case if the turbine sits around 25% efficiency - where does the pump sit?

In [15]:
%%render

eta_p = eta_o/0.25

<IPython.core.display.Latex object>